# **Datos**

In [2]:
#datos
y=[1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,8,9,1,3,4,5,6,8,9,1,4,5,6,8,9]
X=[[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0,0],
[0,1,0,0,0,0,1,0,0],
[0,1,0,0,0,0,1,0,0],
[0,1,0,0,0,0,1,0,0],
[0,1,0,0,0,0,1,0,0],
[0,1,0,0,0,0,1,0,0],
[0,1,0,0,0,0,1,0,0],
[0,1,0,0,0,0,1,0,0],
[0,1,1,0,0,0,1,0,0],
[0,1,1,0,0,0,1,0,0],
[0,1,1,0,0,0,1,0,0],
[0,1,1,0,0,0,1,0,0],
[0,1,1,0,0,0,1,0,0],
[0,1,1,0,0,0,1,0,0]]

# **Funciones de activacion y Etiquetado One Hot**

In [3]:
# libreias a utilizar
import random
import math
# funciones a utilizar
def relu(x):
    return max(0, x)

def relu_derivative(x):
    return 1 if x > 0 else 0

def softmax(x):
    max_x = max(x)
    exp_x = [math.exp(i - max_x) for i in x]
    sum_exp_x = sum(exp_x)
    return [i / sum_exp_x for i in exp_x]

def one_hot_encode(y, num_classes):
    one_hot = [[0] * num_classes for _ in range(len(y))]
    for i, val in enumerate(y):
        one_hot[i][val - 1] = 1
    return one_hot

# **Inicializacion de Pesos, Propagación y Retropropagación**

In [4]:
# Red neuronal feedforward

# Inicialización de pesos
def init_weights(input_size, hidden_size, output_size):
    weights_input_hidden = [[random.uniform(-0.1, 0.1) for _ in range(hidden_size)] for _ in range(input_size)]
    weights_hidden_output = [[random.uniform(-0.1, 0.1) for _ in range(output_size)] for _ in range(hidden_size)]
    biases_hidden = [random.uniform(-0.1, 0.1) for _ in range(hidden_size)]
    biases_output = [random.uniform(-0.1, 0.1) for _ in range(output_size)]
    return weights_input_hidden, weights_hidden_output, biases_hidden, biases_output

# Propagación hacia adelante
def forward_propagation(x, weights_input_hidden, weights_hidden_output, biases_hidden, biases_output):
    hidden_input = [sum(x[i] * weights_input_hidden[i][j] for i in range(len(x))) + biases_hidden[j] for j in range(len(biases_hidden))]
    hidden_output = [relu(h) for h in hidden_input]

    final_input = [sum(hidden_output[j] * weights_hidden_output[j][k] for j in range(len(hidden_output))) + biases_output[k] for k in range(len(biases_output))]
    final_output = softmax(final_input)
    return hidden_output, final_output

# Backpropagation
def backward_propagation(x, y, hidden_output, final_output, weights_input_hidden, weights_hidden_output, biases_hidden, biases_output, learning_rate):
    # Error en la salida
    output_error = [final_output[k] - y[k] for k in range(len(y))]

    # Gradientes de la capa de salida
    grad_output = output_error
    biases_output = [biases_output[k] - learning_rate * grad_output[k] for k in range(len(biases_output))]
    weights_hidden_output = [[weights_hidden_output[j][k] - learning_rate * grad_output[k] * hidden_output[j] for k in range(len(grad_output))] for j in range(len(hidden_output))]

    # Error en la capa oculta
    hidden_error = [sum(output_error[k] * weights_hidden_output[j][k] for k in range(len(output_error))) * relu_derivative(hidden_output[j]) for j in range(len(hidden_output))]

    # Gradientes de la capa oculta
    biases_hidden = [biases_hidden[j] - learning_rate * hidden_error[j] for j in range(len(hidden_error))]
    weights_input_hidden = [[weights_input_hidden[i][j] - learning_rate * hidden_error[j] * x[i] for j in range(len(hidden_error))] for i in range(len(x))]

    return weights_input_hidden, weights_hidden_output, biases_hidden, biases_output

# **Funciones de Entrenamiento y Predicción**

In [5]:
# funciones de entrenamiento y prediccion
# Entrenamiento
def train(X, Y, input_size, hidden_size, output_size, learning_rate, epochs):
    # Inicializar pesos y sesgos
    weights_input_hidden, weights_hidden_output, biases_hidden, biases_output = init_weights(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        total_loss = 0
        for x, y in zip(X, Y):
            # Forward pass
            hidden_output, final_output = forward_propagation(x, weights_input_hidden, weights_hidden_output, biases_hidden, biases_output)

            # Calcular pérdida (cross-entropy)
            loss = -sum(y[k] * math.log(final_output[k] + 1e-15) for k in range(len(y)))
            total_loss += loss

            # Backpropagation
            weights_input_hidden, weights_hidden_output, biases_hidden, biases_output = backward_propagation(
                x, y, hidden_output, final_output, weights_input_hidden, weights_hidden_output, biases_hidden, biases_output, learning_rate
            )

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(X)}")

    return weights_input_hidden, weights_hidden_output, biases_hidden, biases_output

# Predicción
def predict(x, weights_input_hidden, weights_hidden_output, biases_hidden, biases_output):
    _, final_output = forward_propagation(x, weights_input_hidden, weights_hidden_output, biases_hidden, biases_output)
    return final_output.index(max(final_output))

# **Entrenamiento y Predicción**

In [28]:
#convertir y en salida binaria
Y = one_hot_encode(y, 9)

# Entrenar
input_size = 9
hidden_size = 18
output_size = 9
learning_rate = 0.002
epochs = 5000

weights_input_hidden, weights_hidden_output, biases_hidden, biases_output = train(X, Y, input_size, hidden_size, output_size, learning_rate, epochs)

# Predicción de ejemplo
x_test = [1, 0, 0, 0, 0, 0, 0, 0, -1]
prediction = predict(x_test, weights_input_hidden, weights_hidden_output, biases_hidden, biases_output)
print("Predicción:", prediction + 1)  # Ajustar a índice base 1

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Epoch 2/5000, Loss: 2.196068222281471
Epoch 3/5000, Loss: 2.195355593841817
Epoch 4/5000, Loss: 2.194652440917912
Epoch 5/5000, Loss: 2.193958586912361
Epoch 6/5000, Loss: 2.193273859402078
Epoch 7/5000, Loss: 2.192598090007398
Epoch 8/5000, Loss: 2.1919311142663878
Epoch 9/5000, Loss: 2.1912727715141416
Epoch 10/5000, Loss: 2.1906225011902722
Epoch 11/5000, Loss: 2.189981181477758
Epoch 12/5000, Loss: 2.189349023177242
Epoch 13/5000, Loss: 2.1887248309926886
Epoch 14/5000, Loss: 2.1881084639742316
Epoch 15/5000, Loss: 2.1874997842294643
Epoch 16/5000, Loss: 2.186898656836729
Epoch 17/5000, Loss: 2.186304949761555
Epoch 18/5000, Loss: 2.185716664867755
Epoch 19/5000, Loss: 2.185139706338475
Epoch 20/5000, Loss: 2.184570409018461
Epoch 21/5000, Loss: 2.184007912346304
Epoch 22/5000, Loss: 2.183452102765881
Epoch 23/5000, Loss: 2.1829022613256117
Epoch 24/5000, Loss: 2.182358671631302
Epoch 25/5000, Loss: 2.18182333285610

# **Juego Tres en Raya**

In [30]:
def print_board(board):
    """ Función para mostrar el tablero """
    symbols = [' ', 'X', 'O']
    print("\n")
    for i in range(3):
        print(f" {symbols[board[i*3]]} | {symbols[board[i*3+1]]} | {symbols[board[i*3+2]]} ")
        if i < 2:
            print("---|---|---")
    print("\n")

def is_winner(board, player):
    """ Verifica si un jugador ha ganado """
    win_patterns = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
        [0, 4, 8], [2, 4, 6]              # Diagonales
    ]
    return any(all(board[pos] == player for pos in pattern) for pattern in win_patterns)

def is_full(board):
    """ Verifica si el tablero está lleno """
    return all(cell != 0 for cell in board)

def get_valid_moves(board):
    """ Devuelve las posiciones disponibles para jugar """
    return [i for i in range(len(board)) if board[i] == 0]

def get_move(board, player):
    """ Devuelve el movimiento de la IA usando la red neuronal """
    valid_moves = get_valid_moves(board)
    best_move = -1
    best_prob = -float('inf')

    # Probar cada movimiento posible y seleccionar el que da la mejor probabilidad
    for move in valid_moves:
        board_copy = board[:]
        board_copy[move] = player
        prediction = predict(board_copy, weights_input_hidden, weights_hidden_output, biases_hidden, biases_output)
        if prediction > best_prob:
            best_prob = prediction
            best_move = move

    return best_move

# Juego
def play_game():
    """ Función principal para jugar """
    board = [0] * 9  # 0 representa celdas vacías, 1 = 'X', -1 = 'O'
    current_player = 1  # Jugador 1 (IA)

    while True:
        print_board(board)

        if current_player == 1:  # IA juega
            move = get_move(board, current_player)
            board[move] = current_player
            print(f"IA juega en la posición {move+1}")
        else:  # Jugador humano juega
            move = int(input("Tu movimiento (1-9): ")) - 1
            if board[move] != 0:
                print("¡Posición ocupada! Intenta de nuevo.")
                continue
            board[move] = current_player
            print(f"Tu jugada: {move+1}")

        # Comprobar si hay ganador
        if is_winner(board, current_player):
            print_board(board)
            if current_player == 1:
                print("¡La IA gana!")
            else:
                print("¡Has ganado!")
            break

        # Comprobar si hay empate
        if is_full(board):
            print_board(board)
            print("¡Es un empate!")
            break

        # Cambiar de jugador
        current_player = -current_player  # Cambiar entre 1 y -1

# Llamar a la función para iniciar el juego
play_game()




   |   |   
---|---|---
   |   |   
---|---|---
   |   |   


IA juega en la posición 3


   |   | X 
---|---|---
   |   |   
---|---|---
   |   |   


Tu movimiento (1-9): 2
Tu jugada: 2


   | O | X 
---|---|---
   |   |   
---|---|---
   |   |   


IA juega en la posición 5


   | O | X 
---|---|---
   | X |   
---|---|---
   |   |   


Tu movimiento (1-9): 7
Tu jugada: 7


   | O | X 
---|---|---
   | X |   
---|---|---
 O |   |   


IA juega en la posición 1


 X | O | X 
---|---|---
   | X |   
---|---|---
 O |   |   


Tu movimiento (1-9): 9
Tu jugada: 9


 X | O | X 
---|---|---
   | X |   
---|---|---
 O |   | O 


IA juega en la posición 6


 X | O | X 
---|---|---
   | X | X 
---|---|---
 O |   | O 


Tu movimiento (1-9): 4
Tu jugada: 4


 X | O | X 
---|---|---
 O | X | X 
---|---|---
 O |   | O 


IA juega en la posición 8


 X | O | X 
---|---|---
 O | X | X 
---|---|---
 O | X | O 


¡Es un empate!
